In [15]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
import jieba.posseg as pseg
import re



In [16]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            item_str += " "
    return item_str

# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [17]:
# 读取文件
f = open('./weibos.txt', 'r', encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[。！？\n]', text.replace(u'\u200b', '').replace(u'”','').replace(u'“',''))

# 最后一行如果为空，则删除
for i,_ in enumerate(sentences):
    if sentences[i]==''or sentences[i]==' ':
        sentences.pop(i)
#if sentences[len(sentences)-1] == '':
#    sentences.pop()
print(sentences)
#print(len(sentences))

['#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用', '国足输给叙利亚之后，里皮辞职', '谁将成为新主帅，成为广大球迷关注的焦点', '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里', '据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决', '据了解', '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战', '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利', '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话', '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商', '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……', '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行', '斯科拉里想执教中国国足', '老头有点意思，凡是里皮干过的地方，他就想试试', '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛', '可以试试', '斯科拉里的水平，还不如里皮', '斯科拉里，看好的不是国足，而是年薪…… ', '非常应该辞职', '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有', '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了', '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪', '赛后，叙利亚主教练在更衣室里给每个队员一个耳光', '主教练说：赛前老子就再三交代，这一场无论如何都不能赢中国队', '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去', '，球员委屈的说：七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛', '里皮辞职返回意大利，他的助教马达洛尼随队返回广州', '马达洛尼在接受采访时还原了当时更衣室中的情况：当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事', '中国足协：接受里皮辞职

In [18]:
# 设置停用词
stop = [line.strip() for line in open('./stop_words',encoding='UTF-8').readlines()]
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)
print (documents)

['斯科拉里 执教 国足 一届 里 皮 芾 尾 很大 机会 入 世界杯 一届 没 几个 能用 归化 没用 ', '国 足 输给 叙利亚 里 皮 辞职 ', '新 主帅 球迷 关注 焦点 ', '舆论 倾向 三个 山东鲁能 主帅 李霄鹏 武汉 卓尔 主帅 李铁 前 广州 恒大 主帅 斯科拉里 ', '中国足协 态度 里 皮 请辞 意 决 ', '', '比赛 当晚 太太 西蒙内塔 女士 儿子 小里皮 现场 台上 观战 ', '辞职 里皮 改变 原有 计划 赛后 第二天 迪拜 飞 回 意大利 ', '意味着 本来 没 打算 球队 管理层 中国足协 高层 赛后 第一 时间 辞职 对话 ', '辞职 善后工作 包括 合同 沟通 工作 要待 日后 进一步 协商 ', '回顾 国 足 历届 外籍 教练 里 皮 佩兰 卡马乔 杜伊 科维奇 阿里 汉 米卢 … … ', '有名 水 国足 踢 不好 足协 足协 解散 重组 天王老子 请来 不行 ', '斯科拉里 想 执教 中国 国 足 ', '老头 里 皮 干 地方 想 试试 ', '老头 世界杯 冠军 教头 折 中国 没 丢人 里 皮 折 ', '试试 ', '斯科拉里 水平 里 皮 ', '斯科拉里 看好 国 足 年薪 … …   ', '辞职 ', '中国 足球 名帅 外籍 教练 一点儿 毛用 ', '施拉普纳 二十余年 间 中国 足球 大踏步 倒退 一点儿 杀 不住 车 奶奶 刹车 系统 坏 ', '穿着 几百块 钱 球衣 几千块 钱 球鞋 几万块 钱 包 几十万 包机 几百万 千万 年薪 ', '赛后 叙利亚 主教练 更衣室 里 队员 耳光 ', '主教练 说 赛前 老子 再三 交代 一场 无论如何 赢 中国队 ', '中国 援助 粮食 美金 援助 国家 狗日 吃 土 ', '球员 委屈 说 七十多 分钟 晓得 龟儿子 家 球门 踢 ', '里 皮 辞职 返回 意大利 助教 马达 洛尼 随队 返回 广州 ', '马达 洛尼 接受 采访 时 还原 更衣室 中 情况 更衣室 球员 试图 说服 里 皮 收回 队长 郑智 尝试 阻止 足协 代表 希望 考虑一下 建议 重新考虑 无济于事 ', '中国足协 接受 里 皮 辞职 请求 深刻反思 ', '报道 马达 洛尼 说 里 皮 辞职 事先 告知 发生 也许 里 皮 头脑 一热 ',

In [19]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()


In [20]:
query = '中国足协：接受里皮辞职请求，将深刻反思'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

In [21]:
# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), sentences[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

9 0.0859375 至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商
28 1.0 中国足协：接受里皮辞职请求，将深刻反思
4 0.46875 据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决
Top 3 邻居 [9, 28, 4]
